In [1]:
from mlm import *
from fairseq import checkpoint_utils, options, tasks, utils, data, options
import os
import sys
sys.path.append("..")
from models.mlm import *
sys.path.append("..")
from criterions.mlm import *
from fairseq.data import (
    data_utils,
    Dictionary,
    iterators,
    FairseqDataset,
    IdDataset,
    NestedDictionaryDataset,
    NumSamplesDataset,
    NumelDataset,
    PrependTokenDataset,
    SortDataset,
    TokenBlockDataset,
    MaskTokensDataset,
    PadDataset
)

In [2]:
mlm_pretrained_model = '../../../../rna-emb/ITP-RNAcentral-cdhit-rna_mlm_base-MAXLEN1024-ckpt/checkpoint38.pt'

In [3]:
arg_overrides = { "data": '../../../../../../data/RNAcentral_data/rnacentral_data_cdhit_ftvocab_bin/' }

models, args, task = checkpoint_utils.load_model_ensemble_and_task(mlm_pretrained_model.split(os.pathsep), 
                                                                   arg_overrides=arg_overrides)

In [5]:
model = models[0]
model

RNABaseModel(
  (encoder): RNAMaskedLMEncoder(
    (sentence_encoder): TransformerSentenceEncoder(
      (dropout_module): FairseqDropout()
      (embed_tokens): Embedding(25, 768, padding_idx=1)
      (segment_embeddings): Embedding(2, 768)
      (embed_positions): SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0): TransformerSentenceEncoderLayer(
          (dropout_module): FairseqDropout()
          (activation_dropout_module): FairseqDropout()
          (self_attn): MultiheadAttention(
            (dropout_module): FairseqDropout()
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_feature

In [6]:
RNA_SS_data = collections.namedtuple('RNA_SS_data','seq ss_label length name pairs')
train_data = RNASSDataGenerator('../../../../downstream_tasks/ss-rna/data/', 'train_600.pickle')
train_dataset = Dataset_FCN(train_data)
train_generator = data.DataLoader(train_dataset,
        batch_size = 1,
        shuffle = True)

<ipython-input-2-6d95ff3be3ee>:114: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.pairs = np.array([instance[-1] for instance in self.data])


In [18]:
i = 0
for contacts, seq_embeddings, _ , seq_lens, data_seq , _, seq, seq_index in train_generator:
    if i ==1:
        break
    input = seq_index[:,:seq_lens+2].long()
    contacts_true = contacts[:,:seq_lens,:seq_lens]
    print("len:",len(seq[0][:seq_lens]))
    print(seq_index[:,:seq_lens+2].long())
    print(seq[0][:seq_lens])
    print(contacts[:,:seq_lens,:seq_lens])
    i +=1

len: 352
tensor([[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 7, 7, 5, 7, 4, 4, 5,
         5, 5, 7, 4, 5, 7, 4, 4, 6, 4, 7, 7, 4, 4, 5, 6, 5, 5, 6, 4, 6, 7, 7, 4,
         4, 7, 4, 5, 4, 4, 4, 6, 4, 5, 7, 7, 6, 6, 5, 4, 4, 4, 5, 5, 5, 4, 6, 4,
         7, 7, 5, 7, 5, 4, 5, 4, 5, 4, 7, 5, 5, 5, 7, 7, 4, 7, 7, 4, 5, 6, 4, 4,
         6, 7, 7, 6, 6, 6, 5, 5, 4, 4, 4, 5, 7, 5, 7, 5, 4, 4, 7, 5, 5, 4, 4, 4,
         6, 4, 5, 5, 5, 4, 4, 4, 6, 4, 7, 4, 4, 6, 5, 5, 4, 5, 4, 7, 4, 7, 5, 7,
         7, 4, 7, 4, 7, 4, 5, 7, 7, 4, 4, 6, 5, 5, 7, 4, 4, 5, 5, 4, 7, 4, 4, 7,
         5, 5, 4, 4, 7, 5, 5, 5, 7, 7, 7, 7, 5, 7, 7, 7, 4, 4, 5, 4, 7, 5, 5, 4,
         5, 7, 7, 4, 5, 5, 6, 5, 4, 4, 4, 5, 7, 6, 4, 7, 4, 7, 4, 7, 7, 7, 4, 7,
         5, 5, 4, 4, 4, 7, 5, 4, 4, 7, 7, 6, 4, 6, 6, 6, 7, 4, 4, 4, 7, 7, 4, 4,
         7, 5, 4, 6, 7, 7, 4, 4, 4, 6, 6, 4, 4, 6, 7, 4, 7, 6, 6, 4, 5, 4, 7, 4,
         7, 4, 7, 7, 7, 4, 4, 7, 5, 5, 7, 4, 4, 4, 7, 7, 4, 7, 7, 6, 5, 4, 5, 4,
         4, 5, 5, 6

In [29]:
contacts_true[0][45,:]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [4]:
ss_net = rna_ss_resnet32(initial=True)
for _,param in enumerate(ss_net.named_parameters()):
    print(param[0])
    print(param[1])
    print('----------------')

backbone.embed_tokens.weight
Parameter containing:
tensor([[ 0.0361,  0.0512,  0.0435,  ..., -0.0308, -0.0612, -0.2322],
        [ 0.0118,  0.0267,  0.0627,  ..., -0.0796,  0.0286, -0.1277],
        [ 0.0626,  0.0260,  0.0057,  ..., -0.2235,  0.0219, -0.4590],
        ...,
        [ 0.0325,  0.0059,  0.0163,  ..., -0.1283, -0.0948, -0.2063],
        [ 0.0534, -0.0087,  0.0209,  ..., -0.1247, -0.0723, -0.2153],
        [ 0.0393, -0.0023,  0.0100,  ..., -0.1189, -0.1931, -0.1958]],
       requires_grad=True)
----------------
backbone.segment_embeddings.weight
Parameter containing:
tensor([[-0.0185, -0.0047, -0.0200,  ..., -0.0177,  0.0086, -0.0174],
        [ 0.0151, -0.0332, -0.0182,  ...,  0.0189,  0.0191,  0.0327]],
       requires_grad=True)
----------------
backbone.layers.0.self_attn.k_proj.weight
Parameter containing:
tensor([[ 0.0582,  0.0563,  0.0118,  ..., -0.0487,  0.0324, -0.0364],
        [-0.0184,  0.0258, -0.0302,  ...,  0.0449, -0.1060,  0.0672],
        [-0.0448,  0.0006

In [5]:
for _,param in enumerate(model.named_parameters()):
    print(param[0])
    print(param[1])
    print('----------------')

encoder.lm_output_learned_bias
Parameter containing:
tensor([-9.1602e-01, -6.4844e-01, -5.7617e-01, -6.4990e-01,  7.4053e-04,
         1.0902e-02,  9.1934e-03,  1.6937e-03, -1.8298e-01, -4.9292e-01,
        -4.7314e-01, -4.2969e-01, -4.3457e-01, -4.5337e-01, -4.6118e-01,
        -5.1074e-01, -5.4053e-01, -5.5615e-01, -5.6982e-01, -6.4258e-01,
        -9.0967e-01, -9.2090e-01, -9.1699e-01, -9.1211e-01, -2.5879e+00],
       requires_grad=True)
----------------
encoder.sentence_encoder.embed_tokens.weight
Parameter containing:
tensor([[ 0.0361,  0.0512,  0.0435,  ..., -0.0308, -0.0612, -0.2322],
        [ 0.0118,  0.0267,  0.0627,  ..., -0.0796,  0.0286, -0.1277],
        [ 0.0626,  0.0260,  0.0057,  ..., -0.2235,  0.0219, -0.4590],
        ...,
        [ 0.0325,  0.0059,  0.0163,  ..., -0.1283, -0.0948, -0.2063],
        [ 0.0534, -0.0087,  0.0209,  ..., -0.1247, -0.0723, -0.2153],
        [ 0.0393, -0.0023,  0.0100,  ..., -0.1189, -0.1931, -0.1958]],
       requires_grad=True)
---------

In [19]:
task.source_dictionary.symbols

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 'G',
 'A',
 'U',
 'C',
 'N',
 'Y',
 'R',
 'S',
 'K',
 'W',
 'M',
 'D',
 'H',
 'V',
 'B',
 'X',
 'I',
 'madeupword0000',
 'madeupword0001',
 'madeupword0002',
 '<mask>']

In [5]:
args

Namespace(act_dropout=0.0, activation_fn='gelu', adam_betas='(0.9,0.98)', adam_eps=1e-06, add_prev_output_tokens=False, all_gather_list_size=16384, apply_bert_init=True, arch='rna_mlm_base', attention_dropout=0.1, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=1.0, cpu=False, criterion='rna_mlm', curriculum=0, data='../../../../../../data/RNAcentral_data/rnacentral_data_cdhit_ftvocab_bin/', data_buffer_size=10, dataset_impl=None, ddp_backend='no_c10d', device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method='env://', distributed_no_spawn=True, distributed_port=-1, distributed_rank=0, distributed_world_size=24, distributed_wrapper='DDP', dropout=0.1, empty_cache_freq=0, encoder_attention_heads=12, encoder_embed_dim=768, encoder_ffn_embed_dim=3072, encoder_layers=12, encoder_learned_pos=False, encoder_normali

In [6]:
model.prepare_for_inference_(args)
model.to('cpu')
model.eval()

In [30]:
epoch = 1
split = 'valid'
paths = utils.split_paths(task.args.data)
assert len(paths) > 0
data_path = paths[(epoch - 1) % len(paths)]
split_path = os.path.join(data_path, split)
dataset = data_utils.load_indexed_dataset(
            split_path,
            task.source_dictionary,
            task.args.dataset_impl,
            combine=False,
        )

In [14]:
epoch = 1
split = 'valid'
paths = utils.split_paths(task.args.data)
assert len(paths) > 0
data_path = paths[(epoch - 1) % len(paths)]
split_path = os.path.join(data_path, split)
split_path

'../../../../../../data/RNAcentral_data/rnacentral_data_cdhit_ftvocab_bin/valid'

In [31]:
dataset = TokenBlockDataset(
            dataset,
            dataset.sizes,
            task.args.tokens_per_sample - 1,  # one less for <s>
            pad=task.source_dictionary.pad(),
            eos=task.source_dictionary.eos(),
            break_mode=task.args.sample_break_mode,
        )

In [23]:
dataset[12]

tensor([4, 7, 7, 6, 7, 7, 7, 6, 7, 7, 4, 7, 7, 4, 7, 7, 5, 6, 7, 5, 4, 5, 7, 6,
        7, 7, 6, 5, 7, 4, 4, 4, 5, 4, 4, 5, 5, 4, 7, 5, 4, 6, 4, 4, 4, 4, 5, 5,
        6, 6, 6, 6, 4, 4, 4, 5, 7, 6, 5, 5, 6, 4, 4, 4, 4, 4, 7, 5, 5, 7, 7, 7,
        4, 6, 4, 5, 6, 7, 7, 5, 4, 7, 7, 5, 6, 4, 7, 7, 4, 7, 4, 6, 4, 5, 4, 6,
        4, 5, 5, 4, 5, 5, 4, 4, 7, 6, 7, 4, 4, 4, 6, 6, 4, 6, 5, 5, 5, 4, 7, 6,
        7, 6, 6, 6, 7, 5, 4, 7, 4, 4, 4, 5, 7, 5, 5, 5, 5, 7, 4, 4, 4, 7, 5, 7,
        7, 6, 4, 7, 4, 5, 5, 6, 5, 5, 7, 4, 4, 4, 6, 4, 6, 6, 5, 5, 6, 4, 5, 6,
        4, 4, 6, 5, 7, 7, 7, 4, 5, 5, 4, 5, 5, 4, 5, 5, 4, 7, 5, 7, 7, 4, 4, 7,
        6, 5, 5, 7, 6, 5, 7, 4, 6, 4, 7, 7, 5, 4, 7, 7, 4, 7, 7, 4, 7, 4, 4, 6,
        5, 5, 6, 5, 7, 2])

In [32]:
dataset = PrependTokenDataset(dataset, task.source_dictionary.bos())

In [25]:
dataset[12]

tensor([0, 4, 7, 7, 6, 7, 7, 7, 6, 7, 7, 4, 7, 7, 4, 7, 7, 5, 6, 7, 5, 4, 5, 7,
        6, 7, 7, 6, 5, 7, 4, 4, 4, 5, 4, 4, 5, 5, 4, 7, 5, 4, 6, 4, 4, 4, 4, 5,
        5, 6, 6, 6, 6, 4, 4, 4, 5, 7, 6, 5, 5, 6, 4, 4, 4, 4, 4, 7, 5, 5, 7, 7,
        7, 4, 6, 4, 5, 6, 7, 7, 5, 4, 7, 7, 5, 6, 4, 7, 7, 4, 7, 4, 6, 4, 5, 4,
        6, 4, 5, 5, 4, 5, 5, 4, 4, 7, 6, 7, 4, 4, 4, 6, 6, 4, 6, 5, 5, 5, 4, 7,
        6, 7, 6, 6, 6, 7, 5, 4, 7, 4, 4, 4, 5, 7, 5, 5, 5, 5, 7, 4, 4, 4, 7, 5,
        7, 7, 6, 4, 7, 4, 5, 5, 6, 5, 5, 7, 4, 4, 4, 6, 4, 6, 6, 5, 5, 6, 4, 5,
        6, 4, 4, 6, 5, 7, 7, 7, 4, 5, 5, 4, 5, 5, 4, 5, 5, 4, 7, 5, 7, 7, 4, 4,
        7, 6, 5, 5, 7, 6, 5, 7, 4, 6, 4, 7, 7, 5, 4, 7, 7, 4, 7, 7, 4, 7, 4, 4,
        6, 5, 5, 6, 5, 7, 2])

In [33]:
type(dataset)

fairseq.data.prepend_token_dataset.PrependTokenDataset

In [35]:
len(dataset)

2590001

In [36]:
src_dataset, tgt_dataset = MaskTokensDataset.apply_mask(
            dataset,
            task.source_dictionary,
            pad_idx=task.source_dictionary.pad(),
            mask_idx=task.mask_idx,
            seed=task.args.seed,
            mask_prob=task.args.mask_prob,
            leave_unmasked_prob=task.args.leave_unmasked_prob,
            random_token_prob=task.args.random_token_prob,
            freq_weighted_replacement=task.args.freq_weighted_replacement,
            mask_whole_words=None,
        )

In [37]:
len(src_dataset)

2590001

Bad pipe message: %s [b'n\xdep\xf7\xf2\xeb]\x08\xb3"/\x1d\xba\xc0P\xc8\xac|\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0\'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x003\x002\x00\x9a\x00\x99\x00E\x00D\xc0\x07\xc0\x11\xc0\x08\xc0\x12\x00\x16\x00\x13\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00\xc0\x00<\x00\xba\x005\x00\x84\x00/\x00\x96\x00A\x00\x05\x00\n\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00']
Bad pipe message: %s [b'P\x18\xe7\xddeR\x9e\xb4\x85\x9a\xeb\xa2\xcc\x80\xe3T\xa9\xd9\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t\x003\x002\x001\x000\xc0\x0e\xc0\x04\x00/\x00\x9a\x00\x99\x00\x98\x00\x97\x00\x96\x00\x07\xc0\x11\xc0\x07\xc0\x0c\xc0\x02\x00\x05\x00\x04\x00\xff\x02\x01\

In [27]:
src_dataset[12]

tensor([24,  4,  7,  7, 24,  7,  7,  7,  6,  7,  7,  4,  7,  7,  4,  7,  7,  5,
         6,  7, 19,  4,  5,  7,  6,  7,  7, 24,  5,  7,  4,  4,  4,  5,  4,  4,
         5,  5, 24,  7,  5,  4, 24,  4,  4,  4,  4,  5,  5,  6,  6,  6,  6,  4,
         4,  4,  5,  7,  6,  5,  5,  6,  4,  4,  4,  4,  4,  7,  5,  5,  7,  7,
         7, 24,  6,  4,  5,  6, 24,  7,  4,  4,  7,  7,  5, 24,  4,  7,  7, 24,
        24,  4,  6,  4,  5,  4,  6,  4, 24,  5,  4,  5,  5,  4,  4,  7,  6,  7,
         4,  4, 24,  6,  6,  4,  6,  5,  5,  5, 24,  7,  6,  7,  6,  6,  6,  7,
         5,  4,  4,  4,  4,  4, 24,  7,  5,  5,  5,  5,  7,  4, 24,  4,  7,  5,
        24,  7,  6,  4,  7,  7,  5,  5,  6,  5,  5,  7,  4,  4,  4,  6,  4,  6,
         6,  5, 24,  6,  4, 20,  6,  4,  4,  6,  5,  7,  7,  7,  4, 17,  5,  4,
         5,  5, 24,  5,  5, 24,  7,  5,  7,  7,  4,  4,  7,  6,  5,  5,  7,  6,
         5,  7,  4,  6,  4, 24,  7, 24,  4,  7,  7, 24,  7,  7,  4,  7, 24,  4,
         6,  5,  5,  6,  5,  7, 20])

In [28]:
tgt_dataset[12]

tensor([0, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 5, 1, 1, 1,
        1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 6, 1, 4, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 4, 1, 1, 1, 1, 7, 1, 5, 1, 1, 1, 1, 6, 1, 1, 1, 4, 7, 1, 1, 4, 1, 1,
        1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 4, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1,
        7, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 5,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 4, 1, 1, 4, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 5, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1,
        1, 1, 1, 1, 1, 1, 2])

In [29]:
type(tgt_dataset)

fairseq.data.lru_cache_dataset.LRUCacheDataset

In [14]:
src_dataset[2]

tensor([0, 5, 6,  ..., 7, 6, 2])

In [15]:
kk = PadDataset(
                    src_dataset,
                    pad_idx=task.source_dictionary.pad(),
                    left_pad=False,
                )

In [18]:
kk[3].shape

torch.Size([499])

In [11]:
netinput_dataset = {
            'id': IdDataset(),
            'net_input': {
                'src_tokens': PadDataset(
                    src_dataset,
                    pad_idx=task.source_dictionary.pad(),
                    left_pad=False,
                ),
                'src_lengths': NumelDataset(src_dataset, reduce=False),
            },
            'target': PadDataset(
                tgt_dataset,
                pad_idx=task.source_dictionary.pad(),
                left_pad=False,
            ),
            'nsentences': NumSamplesDataset(),
            'ntokens': NumelDataset(src_dataset, reduce=True),
        }

In [61]:
query_smile_index = 6122
# netinput_dataset = {
#             'id': IdDataset(),
#             'net_input': {
#                 'src_tokens': PadDataset(
#                     src_dataset,
#                     pad_idx=task.source_dictionary.pad(),
#                     left_pad=False,
#                 ),
#                 'src_lengths': NumelDataset(src_dataset, reduce=False),
#             },
#             'target': PadDataset(
#                 tgt_dataset,
#                 pad_idx=task.source_dictionary.pad(),
#                 left_pad=False,
#             ),
#             'nsentences': NumSamplesDataset(),
#             'ntokens': NumelDataset(src_dataset, reduce=True),
#         }
src_input = netinput_dataset['net_input']['src_tokens'][query_smile_index]
print("len:\n",len(netinput_dataset['net_input']['src_tokens'][query_smile_index].detach().numpy()))
print("input:\n",netinput_dataset['net_input']['src_tokens'][query_smile_index].detach().numpy())
target_input = netinput_dataset['target'][query_smile_index]
print("label:\n",netinput_dataset['target'][query_smile_index].detach().numpy())
seq_len = netinput_dataset['net_input']['src_lengths'][query_smile_index]
criterion = task.build_criterion(args)


extra_only = False
masked_only = True
masked_tokens = target_input.ne(criterion.padding_idx)
sample_size = masked_tokens.int().sum()
# Rare: when all tokens are masked, project all tokens.
# We use torch.where to avoid device-to-host transfers,
# except on CPU where torch.where is not well supported
# (see github.com/pytorch/pytorch/issues/26247).
if masked_tokens.device == torch.device('cpu'):
    if not masked_tokens.any():
        masked_tokens = None
else:
    masked_tokens = torch.where(
        masked_tokens.any(),
        masked_tokens,
        masked_tokens.new([True]),
    )

targets = target_input
if masked_tokens is not None:
    targets = targets[masked_tokens]
print("true:",targets.detach().numpy())
logits, _ = model(src_input.reshape((1,seq_len)), masked_tokens=masked_tokens.reshape((1,seq_len)), 
                    masked_only=masked_only, extra_only=extra_only)
print("predict:",np.argmax(logits.detach().numpy(),axis=1))
print("accuracy:",np.sum(np.argmax(logits.detach().numpy(),axis=1) == targets.detach().numpy())/len(targets))

len:
 487
input:
 [ 0 24  5  6  6 24  5 24  7  4  7  6  4  4  7 24  4  7  5  4  4  7 24  6
  5  5  7  5  7  5  6  4  7  5  5  4  6  7  4  5  4  7  4  4  5  6  5  5
 24  4  4 24  5  4  7  6  6  4 20  6  7  7  6  4  4  5  6  6 24  5  4  7
  4  4  7  4  9  5  7  4  4  4 24 24  5  4  6  5  5  6  4  7  7  6  5  4
  4  5  5  6  7  6  4  7  7 24  4  4  6  5  4  6  4  4  4  4 24  5  6  5
  5  7  4  6  7  7 24  4  5 24  5  7  4 24  4  7  4  7  6 24  5  6  5 24
  7  4  7  5  6  5  7  4  6  7 24  6  4  5  4  4  4  5  4  5 24  5 24  6
  4  4  4  4  4  5  6  7  6  6  7 24  4  5  7  7 24  7  5 24  4  7  6  5
  6  7  5  4  5  6 24  5  4  7 24  6  5  4  4  6  7  4 24 17 24  6  5  4
 24 24  5  4  6  6  4  4  6  4  4  4  4  6  5  5  5 24  4  7  7  6  5  7
 24  5  5 24  4  7  4  5  7  4  5  6  7  7  4  6  5  5  7  6  4  4  6  7
  6 24  5  4  5 24  4  5  6  4  5  6  7  5  4  6  9  5  7 24  7  6  4  4
  5  5  7  6  4 24  4  5  7  5  7  4  4  6 24  7  5  4  5 24  6  7  7  6
  5  7  4  4  4  5  4  4  7  5  4

tensor([0, 0, 4,  ..., 7, 5, 2])

In [12]:
# valid mlm acc
valid_input = netinput_dataset['net_input']['src_tokens']
valid_target = netinput_dataset['target']
acc = []
extra_only = False
masked_only = True
for i in range(2):
    src_input = valid_input[i]
    seq_len = len(src_input)
    target_input = valid_target[i]
    masked_tokens = target_input.ne(1)

    if masked_tokens is not None:
        targets = target_input[masked_tokens]
        logits, _ = model(src_input.reshape((1,seq_len)), masked_tokens=masked_tokens.reshape((1,seq_len)), 
                    masked_only=masked_only, extra_only=extra_only)
    acc.append(np.sum(np.argmax(logits.detach().numpy(),axis=1) == targets.detach().numpy())/len(targets))
    

: 

: 

In [49]:
query = 22
src_input = netinput_dataset['net_input']['src_tokens'][query]
seq_len = netinput_dataset['net_input']['src_lengths'][query]
logits, inner = model(src_input.reshape((1,seq_len)),
                    masked_only=False, extra_only=extra_only)

In [57]:
# 第一层是embedding的
len(inner['inner_states'])

13

In [23]:
task.source_dictionary.symbols

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 'A',
 'G',
 'U',
 'C',
 'N',
 'Y',
 'R',
 'S',
 'K',
 'W',
 'M',
 'D',
 'H',
 'V',
 'B',
 'X',
 'I',
 'madeupword0000',
 'madeupword0001',
 'madeupword0002',
 '<mask>']